In [17]:
import sys
sys.path.append('/usr/local/bin/python2.7')


import numpy as np
import os, sys, getopt, pickle, csv, sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix, make_scorer, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from textblob import TextBlob
import random
import matplotlib.pyplot as plt
from sklearn import metrics
from collections import Counter
import argparse
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score    
import preprocessor as p

In [18]:
models = [ 'svm', 'naive', 'lr', 'random_forest']
NO_OF_FOLDS = 10
MODEL_TYPE = "all"
HASH_REMOVE = None
LABEL_ENCODING_TWITTER = {'racism':0,'sexism':1,'none':2}
LABEL_ENCODING_FORMSPRING = {'none':0,'bully':1}
LABEL_ENCODING_WIKI = {'none':0,'attack':1}

In [19]:
def load_data(filename):
    data = pickle.load(open(filename, 'rb'))
    x_text = []
    labels = []
    for i in range(len(data)):
        if(HASH_REMOVE):
            x_text.append(p.tokenize((data[i]['text']).encode('utf-8')))
        else:
            x_text.append(data[i]['text'])
        labels.append(data[i]['label'])
    return x_text,labels

def get_filename(dataset):
    global N_CLASS, HASH_REMOVE, LABEL_ENCODING
    if(dataset=="twitter"):
        filename = "data/twitter_data.pkl"
        N_CLASS = 3
        LABEL_ENCODING = LABEL_ENCODING_TWITTER
        HASH_REMOVE = False
    elif(dataset=="formspring"):
        N_CLASS = 2
        LABEL_ENCODING = LABEL_ENCODING_FORMSPRING
        filename = "data/formspring_data.pkl"
        HASH_REMOVE = False
    elif(dataset=="wiki"):
        N_CLASS = 2
        LABEL_ENCODING = LABEL_ENCODING_WIKI
        filename = "data/wiki_data.pkl"
        HASH_REMOVE = False
    return filename

In [20]:
def get_scores(y_true, y_pred):
    #dict1 = {'racism':0,'sexism':1,'none':2}
    if(data == "twitter"):
        scores = np.array([ 
                    precision_score(y_true, y_pred, average=None, labels=[2, 0, 1]), 
                    recall_score(y_true, y_pred,  average=None, labels=[2, 0, 1]),
                    f1_score(y_true, y_pred, average=None, labels=[2, 0, 1])])
    else:
        scores = np.array([ 
                    precision_score(y_true, y_pred, average=None), 
                    recall_score(y_true, y_pred,  average=None),
                    f1_score(y_true, y_pred, average=None)])
    return scores
    
def print_scores(scores):
    for i in range(N_CLASS):
        scores_class_prec = []
        scores_class_rec = []
        scores_class_f1 = []
        for foldscore in scores:
            scores_class_prec.append(foldscore[i])
            scores_class_rec.append(foldscore[i])
            scores_class_f1.append(foldscore[i])
        scores_class_prec = np.array(scores_class_prec)
        scores_class_rec = np.array(scores_class_rec)
        scores_class_f1 = np.array(scores_class_f1)
        class_string = ""
        for key, value in LABEL_ENCODING.items():
            if(value == i):
                class_string = key
        print "Precision Class %s (avg): %0.3f (+/- %0.3f)" % (class_string,scores_class_prec.mean(), scores_class_prec.std() * 2)
        print "Recall Class %s (avg): %0.3f (+/- %0.3f)" % (class_string,scores_class_rec.mean(), scores_class_rec.std() * 2)
        print "F1_score Class %s (avg): %0.3f (+/- %0.3f)" % (class_string,scores_class_f1.mean(), scores_class_f1.std() * 2)


In [21]:
arr = []
arr.append(np.hstack(np.array(["precision","recall","F1"])))
arr.append(np.hstack(np.array(["precision","recall","F1"])))
arr.append(np.hstack(np.array(["precision","recall","F1"])))
arr = np.array(arr)
print "%s" % str(arr[:, 1])

['recall' 'recall' 'recall']


In [22]:
def classification_model(X, Y, model_type):
    X, Y = shuffle(X, Y, random_state=42)
    print "Model Type:", model_type
    kf = KFold(n_splits=NO_OF_FOLDS)
    scores = []
    for train_index, test_index in kf.split(X):
        Y = np.asarray(Y)
        model = get_model(model_type)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        curr_scores = get_scores(y_test, y_pred) 
        #[
        #   [precision, precision, precision], -> length == number of classes
        #   [recall, recall, recall], 
        #   [f1, f1, f1]
        #]
        scores.append(curr_scores)
        # [
        #   [precision, ... , precision] -> len == number of folds
        #   [recall, ... , recall]
        #   [f1, ... , f1]
        # ]
    print_scores(np.array(scores))

In [23]:
def get_model(m_type):
    if m_type == 'lr':
        logreg = LogisticRegression(class_weight="balanced")
    elif m_type == 'naive':
        logreg =  MultinomialNB()
    elif m_type == "random_forest":
        logreg = RandomForestClassifier(n_estimators=100, n_jobs=-1)
    elif m_type == "svm":
        logreg = LinearSVC(class_weight="balanced")
    else:
        print "ERROR: Please specify a correst model"
        return None
    return logreg

In [24]:
def train(x_text, labels, MODEL_TYPE):
    
    if(WORD):
        print("Using word based features")
        bow_transformer = CountVectorizer(analyzer="word",max_features = 10000,stop_words='english').fit(x_text)
        comments_bow = bow_transformer.transform(x_text)
        tfidf_transformer = TfidfTransformer(norm = 'l2').fit(comments_bow)
        comments_tfidf = tfidf_transformer.transform(comments_bow)
        features = comments_tfidf
    else: 
        print("Using char n-grams based features")
        bow_transformer = CountVectorizer(max_features = 10000, ngram_range = (1,2)).fit(x_text)
        comments_bow = bow_transformer.transform(x_text)
        tfidf_transformer = TfidfTransformer(norm = 'l2').fit(comments_bow)
        comments_tfidf = tfidf_transformer.transform(comments_bow)
        features = comments_tfidf
    
    if(data == "twitter"):
        labels = np.array([LABEL_ENCODING_TWITTER[b] for b in labels])
    
    from collections import Counter
    print(Counter(labels))
    
    if(MODEL_TYPE != "all"):
        classification_model(features, labels, MODEL_TYPE)
    else:
        for model_type in models:
            classification_model(features, labels, model_type)

In [25]:
data = "formspring"
WORD =  False
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using char n-grams based features
Counter({0: 11997, 1: 776})
Model Type: svm
Precision Class 1 (avg): 0.466 (+/- 0.109)
Recall Class 1 (avg): 0.503 (+/- 0.122)
F1_score Class 1 (avg): 0.483 (+/- 0.104)
Model Type: naive
Precision Class 1 (avg): 0.850 (+/- 0.640)
Recall Class 1 (avg): 0.015 (+/- 0.015)
F1_score Class 1 (avg): 0.030 (+/- 0.028)
Model Type: lr


/home/paul/.conda/envs/cyberbullying/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/paul/.conda/envs/cyberbullying/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision Class 1 (avg): 0.410 (+/- 0.099)
Recall Class 1 (avg): 0.626 (+/- 0.131)
F1_score Class 1 (avg): 0.495 (+/- 0.104)
Model Type: random_forest
Precision Class 1 (avg): 0.775 (+/- 0.196)
Recall Class 1 (avg): 0.168 (+/- 0.064)
F1_score Class 1 (avg): 0.274 (+/- 0.087)


In [26]:
data = "formspring"
WORD = True
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using word based features
Counter({0: 11997, 1: 776})
Model Type: svm
Precision Class 1 (avg): 0.415 (+/- 0.089)
Recall Class 1 (avg): 0.525 (+/- 0.132)
F1_score Class 1 (avg): 0.463 (+/- 0.100)
Model Type: naive
Precision Class 1 (avg): 0.575 (+/- 0.950)
Recall Class 1 (avg): 0.013 (+/- 0.029)
F1_score Class 1 (avg): 0.025 (+/- 0.055)
Model Type: lr
Precision Class 1 (avg): 0.407 (+/- 0.079)
Recall Class 1 (avg): 0.617 (+/- 0.127)
F1_score Class 1 (avg): 0.489 (+/- 0.084)
Model Type: random_forest
Precision Class 1 (avg): 0.696 (+/- 0.304)
Recall Class 1 (avg): 0.162 (+/- 0.075)
F1_score Class 1 (avg): 0.261 (+/- 0.112)


In [27]:
data = "twitter"
WORD = False
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using char n-grams based features
Counter({2: 11036, 1: 3117, 0: 1937})
Model Type: svm
Precision Class 1 (avg): 0.731 (+/- 0.057)
Recall Class 1 (avg): 0.783 (+/- 0.063)
F1_score Class 1 (avg): 0.755 (+/- 0.038)
Precision Class 2 (avg): 0.786 (+/- 0.055)
Recall Class 2 (avg): 0.736 (+/- 0.067)
F1_score Class 2 (avg): 0.759 (+/- 0.040)
Model Type: naive
Precision Class 1 (avg): 0.790 (+/- 0.063)
Recall Class 1 (avg): 0.537 (+/- 0.058)
F1_score Class 1 (avg): 0.638 (+/- 0.046)
Precision Class 2 (avg): 0.910 (+/- 0.056)
Recall Class 2 (avg): 0.455 (+/- 0.089)
F1_score Class 2 (avg): 0.605 (+/- 0.082)
Model Type: lr
Precision Class 1 (avg): 0.737 (+/- 0.058)
Recall Class 1 (avg): 0.781 (+/- 0.068)
F1_score Class 1 (avg): 0.758 (+/- 0.050)
Precision Class 2 (avg): 0.825 (+/- 0.048)
Recall Class 2 (avg): 0.641 (+/- 0.084)
F1_score Class 2 (avg): 0.721 (+/- 0.059)
Model Type: random_forest
Precision Class 1 (avg): 0.789 (+/- 0.076)
Recall Class 1 (avg): 0.695 (+/- 0.082)
F1_scor

In [28]:
data = "twitter"
WORD = True
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using word based features
Counter({2: 11036, 1: 3117, 0: 1937})
Model Type: svm
Precision Class 1 (avg): 0.735 (+/- 0.081)
Recall Class 1 (avg): 0.786 (+/- 0.067)
F1_score Class 1 (avg): 0.758 (+/- 0.043)
Precision Class 2 (avg): 0.803 (+/- 0.044)
Recall Class 2 (avg): 0.744 (+/- 0.052)
F1_score Class 2 (avg): 0.772 (+/- 0.037)
Model Type: naive
Precision Class 1 (avg): 0.797 (+/- 0.077)
Recall Class 1 (avg): 0.529 (+/- 0.068)
F1_score Class 1 (avg): 0.635 (+/- 0.062)
Precision Class 2 (avg): 0.904 (+/- 0.035)
Recall Class 2 (avg): 0.469 (+/- 0.056)
F1_score Class 2 (avg): 0.617 (+/- 0.051)
Model Type: lr
Precision Class 1 (avg): 0.741 (+/- 0.065)
Recall Class 1 (avg): 0.786 (+/- 0.056)
F1_score Class 1 (avg): 0.762 (+/- 0.042)
Precision Class 2 (avg): 0.832 (+/- 0.039)
Recall Class 2 (avg): 0.663 (+/- 0.083)
F1_score Class 2 (avg): 0.738 (+/- 0.062)
Model Type: random_forest
Precision Class 1 (avg): 0.774 (+/- 0.057)
Recall Class 1 (avg): 0.746 (+/- 0.084)
F1_score Class 

In [29]:
data = "wiki"
WORD = False
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using char n-grams based features
Counter({0: 102274, 1: 13590})
Model Type: svm
Precision Class 1 (avg): 0.591 (+/- 0.025)
Recall Class 1 (avg): 0.823 (+/- 0.019)
F1_score Class 1 (avg): 0.688 (+/- 0.018)
Model Type: naive
Precision Class 1 (avg): 0.839 (+/- 0.010)
Recall Class 1 (avg): 0.554 (+/- 0.028)
F1_score Class 1 (avg): 0.667 (+/- 0.021)
Model Type: lr
Precision Class 1 (avg): 0.602 (+/- 0.024)
Recall Class 1 (avg): 0.845 (+/- 0.022)
F1_score Class 1 (avg): 0.703 (+/- 0.017)
Model Type: random_forest
Precision Class 1 (avg): 0.888 (+/- 0.017)
Recall Class 1 (avg): 0.549 (+/- 0.028)
F1_score Class 1 (avg): 0.678 (+/- 0.024)


In [30]:
data = "wiki"
WORD = True
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using word based features
Counter({0: 102274, 1: 13590})
Model Type: svm
Precision Class 1 (avg): 0.591 (+/- 0.025)
Recall Class 1 (avg): 0.819 (+/- 0.028)
F1_score Class 1 (avg): 0.686 (+/- 0.020)
Model Type: naive
Precision Class 1 (avg): 0.898 (+/- 0.016)
Recall Class 1 (avg): 0.521 (+/- 0.036)
F1_score Class 1 (avg): 0.659 (+/- 0.028)
Model Type: lr
Precision Class 1 (avg): 0.620 (+/- 0.027)
Recall Class 1 (avg): 0.834 (+/- 0.024)
F1_score Class 1 (avg): 0.711 (+/- 0.021)
Model Type: random_forest
Precision Class 1 (avg): 0.811 (+/- 0.022)
Recall Class 1 (avg): 0.661 (+/- 0.029)
F1_score Class 1 (avg): 0.729 (+/- 0.023)
